In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import fitsio
from astropy.io import fits
import pandas as pd 
import sys
from quickbin import bin2d
from pyarrow import parquet
sys.path.append('/home/bekah/gPhoton2')

In [ ]:
def centile_clip(image, centiles=(1, 99.5)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [ ]:
eclipse = 44405
#star_file = "/home/bekah/gPhoton2/test_data/e44405/e44405-nd-f0120-b00-movie-photom-12_8.csv"
#stars = pd.read_csv(star_file)le

In [ ]:
leg_data = parquet.read_table("/home/bekah/gPhoton2/test_data/e44405/e44405-nd-b00.parquet",
                           columns=['flags','ra','dec','col','row','mask','ya'],filters=[('col','<=',800),('row','<=',800),('col','>=',0),('row','>=',0)]).to_pandas()


In [ ]:
ra, dec, mask, col, row, flags = (
            leg_data["ra"],
            leg_data["dec"],
            leg_data["mask"],
            leg_data["col"],
            leg_data["row"],
            leg_data["flags"],
        )

valid_indices = (col >= 0) & (col <= 800) & (row >= 0) & (row <= 800)
    # have to mask all NaNs for bin2d to work properly
nan_free_indices = valid_indices & ~np.isnan(ya) & ~np.isnan(ra) & ~np.isnan(dec)
ra_valid, dec_valid, flag_valid, mask_valid = ra[nan_free_indices], dec[nan_free_indices], flags[nan_free_indices], mask[nan_free_indices]


ra_valid = np.asarray(ra_valid, dtype=np.float64)
dec_valid = np.asarray(dec_valid, dtype=np.float64)
flag_valid = np.asarray(flag_valid, dtype=np.float64)
mask_valid = np.asarray(mask_valid, dtype=np.float64)

result_flags = bin2d(ra_valid, dec_valid, flag_valid, ('mean','count'), n_bins=1600)
result_mask = bin2d(ra_valid, dec_valid, mask_valid, ('mean','count'), n_bins=1600)

In [ ]:
result = bin2d(photon['col'], photon['row'], photon['ya'], ('std', 'mean','count'), n_bins=1600)
#result_flags = bin2d(leg_data['ra'], leg_data['dec'], leg_data['ya'], ('mean','count'), n_bins=1600)
#result_mask = bin2d(leg_data['ra'], leg_data['dec'], leg_data['row'],g_d ('mean','count'), n_bins=1600)
#resultr = bin2d(photon['ra'], photon['dec'], photon['xa'], ('mean'), n_bins=1600)
#result_ya = bin2d(photon['ra'], photon['dec'], photon['ya'], ('mean'), n_bins=1600)
#result_m = bin2d(photon['ra'], photon['dec'], photon['ya'], ('median'), n_bins=1600)


In [ ]:
plt.scatter(leg_data['mask'], leg_data["flags"],s=.01)
plt.xlabel("mask")
plt.ylabel("flags")

In [ ]:
from quickbin import bin2d
from numpy.lib import recfunctions as rfn

n_bins = 1600

ra, dec, ya, col, row, flags = (
        leg_data["ra"],
        leg_data["dec"],
        leg_data["ya"],
        leg_data["col"],
        leg_data["row"],
        leg_data["flags"],
    )

print("filtering")
valid_indices = (col >= 0) & (col <= 800) & (row >= 0) & (row <= 800)
ra_valid, dec_valid, ya_valid = ra[valid_indices], dec[valid_indices], ya[valid_indices]

print("binning")
binned = bin2d(ra_valid, dec_valid, ya_valid, ('mean', 'count'), n_bins=n_bins)

filtered = (binned['mean'] < 9).astype(int) & (binned['count'] > 15).astype(int)
flagged_pixels = np.where(filtered == 1)

print(len(flagged_pixels))
print(flagged_pixels)

# map binned coordinates to ra/dec values
ra_min, ra_max = np.min(ra_valid), np.max(ra_valid)
dec_min, dec_max = np.min(dec_valid), np.max(dec_valid)
ra_edges = np.linspace(ra_min, ra_max, n_bins + 1)
dec_edges = np.linspace(dec_min, dec_max, n_bins + 1)
ra_flagged = (ra_edges[flagged_pixels[0]] + ra_edges[flagged_pixels[0] + 1]) / 2
dec_flagged = (dec_edges[flagged_pixels[1]] + dec_edges[flagged_pixels[1] + 1]) / 2
print("masking")
mask = np.zeros_like(ra, dtype=bool)


In [ ]:
len(flagged_pixels[0])

In [ ]:
for r, d in zip(ra_flagged, dec_flagged):
    print(r)
    mask |= ((ra >= r) & (ra < r + (ra_max - ra_min) / n_bins)) & \
            ((dec >= d) & (dec < d + (dec_max - dec_min) / n_bins))
print("recarray")
print(len(mask))
leg_data["flags"][mask] = 250

In [ ]:
plt.scatter(ra, dec, c=mask)

In [ ]:
eclipse = 'e44405'
#hdu1 = fits.PrimaryHDU(result['mean'].data)
hdu2 = fits.PrimaryHDU(result_flags['mean'])
hdulist = fits.HDUList([hdu2])
hdulist.writeto(f'e{eclipse}_flags', overwrite=True)
print(f"done w {eclipse}")
eclipse = 'e44405'
#hdu1 = fits.PrimaryHDU(result['mean'].data)
hdu2 = fits.PrimaryHDU(result_mask['mean'])
hdulist = fits.HDUList([hdu2])
hdulist.writeto(f'e{eclipse}_mask', overwrite=True)
print(f"done w {eclipse}")

In [ ]:
eclipse = 'e44405'
#hdu1 = fits.PrimaryHDU(result['mean'].data)
hdu2 = fits.PrimaryHDU(resultya['mean'])
hdulist = fits.HDUList([hdu2])
hdulist.writeto(f'e{eclipse}_ra_dec_{x}', overwrite=True)
print(f"done w {eclipse}")

hdu3 = fits.PrimaryHDU(result['mean'])
hdulist = fits.HDUList([hdu3])
hdulist.writeto(f'e{eclipse}_col_row_{x}', overwrite=True)
print(f"done w {eclipse}")

In [ ]:
plt.imshow(resultr>15)
plt.colorbar()

In [ ]:
filtered = (resultya['mean']<9).astype(int) + (resultya['count']>15).astype(int)
plt.imshow(filtered==2)
plt.colorbar()

In [ ]:
mask = filtered==2 
masked = np.where(mask, np.nan, resultya['count'])

In [ ]:
plt.imshow(centile_clip(masked),cmap='cividis')

In [ ]:
eclipse = 'e44405'
hdu1 = fits.PrimaryHDU(masked)
hdulist = fits.HDUList([hdu1])
hdulist.writeto(f'e{eclipse}_mask', overwrite=True)
print(f"done w {eclipse}")

In [ ]:
len(leg_data)

In [ ]:
plt.hist(np.ravel(leg_data['flags']),bins=100)

In [ ]:
plt.imshow(centile_clip(resultya['mean'],(20,70)),cmap='grey')

In [ ]:
plt.imshow(centile_clip(resultya['count'],(1,95)))

In [ ]:
hdu1 = fits.PrimaryHDU(result['count{'])
hdulist = fits.HDUList([hdu1])
hdulist.writeto(f'e{eclipse}_proj', overwrite=True)
print(f"done w {eclipse}")

In [ ]:
plt.scatter(photon['col'],photon['row'],c=photon['ya'],s=.01)

In [ ]:
aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
metadata  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()

In [ ]:
plt.scatter(aspect['ra'],aspect['dec'],c=aspect['roll'])
plt.colorbar()

In [ ]:
plt.hist(aspect['roll'])

In [ ]:
good_stars = stars[stars['aperture_sum_mask']==0.0]
good_stars = good_stars[good_stars['aperture_sum_edge']<50.0]
good_stars = good_stars[good_stars['aperture_sum']<3500.0]

In [ ]:
plt.scatter(good_stars['xcentroid'],good_stars['ycentroid'],c=good_stars['aperture_sum_0'],s=.5)
plt.colorbar()

In [ ]:
good_stars

In [ ]:
360-34-34

In [ ]:
good_stars.keys()

In [ ]:
plt.hist(good_stars['aperture_sum'],bins=100)

In [ ]:
wig = fits.open("/home/bekah/gPhoton2/gPhoton/cal_data/WIG2_Sep2010.fits")
walk = fits.open("/home/bekah/gPhoton2/gPhoton/cal_data/WLK2_Sep2010.fits")

In [ ]:
from astropy.table import Table

wiggle = Table(wig[1].data).to_pandas()

In [ ]:
plt.scatter(wig[1].data['YA'],wig[1].data['ycor'],c=wig[1].data['YA'],s=.05)
plt.xlabel('ya')
plt.ylabel('y correction in arcseconds') 

In [ ]:
wig[0].header

In [ ]:
plt.scatter(wiggle['ycor'],wiggle.index,c=wiggle['ycor'],s=.05)
plt.xlabel('YA')
plt.ylabel('index') 
plt.colorbar()